# 3.0.0 Export

- En este archivo se exportara la informacion necesaria para el dash

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [23]:
import sys
sys.path.append("../") 

from utils.utils import tipo1
from utils.RFM import rfm_v1, rfm_v2
from utils.paths2 import direcciones_cursos

G_raw, G_processed, G_interim, G_external, G_models, G_reports, G_reports_figures = direcciones_cursos('australian')

In [15]:
df = pd.read_csv(G_raw/'sample.csv')
df['date'] =  pd.to_datetime(df['date'])
df.head()

,invoice,date,customer_id,stockcode,price
0,1,2020-07-25,d18734,y7,50.45
1,2,2020-01-17,c21086,x7,25.30
2,3,2019-07-05,d18185,z5,18.40
3,4,2019-02-26,c18331,z2,5.50
4,5,2019-02-10,b16309,y7,18.40


In [16]:
df_rfm = pd.read_parquet(G_models/'df_rfm.parquet.gzip')
df_ks = pd.read_parquet(G_models/'df_ks.parquet.gzip')

In [17]:
df.shape, df_rfm.shape, df_ks.shape

((100000, 5), (33004, 9), (33004, 3))

In [18]:
# joins
df = pd.merge(df, df_rfm, how='left', on=['customer_id'])  
df = pd.merge(df, df_ks, how='left', on=['customer_id'])
df.shape

(100000, 15)

In [19]:
df.insert(0, 'contador', 1)
df.insert(5, 'stockcode_family', 99)

lc = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']
df['stockcode_family'] = np.where(df['stockcode'].isin(lc), 'X', df['stockcode_family'])

lc = ['y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8']
df['stockcode_family'] = np.where(df['stockcode'].isin(lc), 'Y', df['stockcode_family'])

lc = ['z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8']
df['stockcode_family'] = np.where(df['stockcode'].isin(lc), 'Z', df['stockcode_family'])
df.head()

,contador,invoice,date,customer_id,stockcode,stockcode_family,price,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,rfm_score,group,Cluster,Cluster_medoids
0,1,1,2020-07-25,d18734,y7,Y,50.45,153,5,169.84,3,4,5,4.000000,7,0,0
1,1,2,2020-01-17,c21086,x7,X,25.30,10,3,81.25,5,3,4,4.000000,7,2,3
2,1,3,2019-07-05,d18185,z5,Z,18.40,141,7,94.26,3,4,4,3.666667,6,0,0
3,1,4,2019-02-26,c18331,z2,Z,5.50,40,3,56.10,5,3,3,3.666667,6,2,3
4,1,5,2019-02-10,b16309,y7,Y,18.40,455,2,23.90,1,2,1,1.333333,1,1,2


In [20]:
# Serie de tiempo
df_ts = df.copy()
df_ts = df_ts.loc[:,['date','contador','price']]#,'customer_id','price'
df_ts = df_ts.groupby('date').sum().reset_index()
df_ts.shape

(731, 3)

In [21]:
# Serie de tiempo
df_cus = df.copy()
df_cus = df_cus.loc[:,['date','customer_id','contador']]#'price'
df_cus = df_cus.drop_duplicates()
df_cus = df_cus.groupby(['date']).sum().reset_index()
df_cus.shape

(731, 2)

In [31]:
df_cir = df.copy()
df_cir = df_cir.loc[:,['date','stockcode']]
df_cir = tipo1(df_cir, 'stockcode')
df_cir.shape

(24, 3)

In [33]:
# Serie de tiempo
df_sto = df.copy()
df_sto = df_sto.loc[:,['stockcode','price','contador']]
df_sto = df_sto.groupby(['stockcode']).sum().reset_index()
df_sto = df_sto.sort_values(by=['price'], ascending=False)
df_sto.shape

(24, 3)

In [34]:
# save
df.to_parquet(G_processed/'data.parquet.gzip', compression='gzip')
df_ts.to_parquet(G_processed/'df_ts.parquet.gzip', compression='gzip')
df_cus.to_parquet(G_processed/'df_cus.parquet.gzip', compression='gzip')
df_cir.to_parquet(G_processed/'df_cir.parquet.gzip', compression='gzip')
df_sto.to_parquet(G_processed/'df_sto.parquet.gzip', compression='gzip')
df_ts.to_parquet('../dashboard/data/df_ts.parquet.gzip', compression='gzip')
df_cir.to_parquet('../dashboard/data/df_cir.parquet.gzip', compression='gzip')
df_sto.to_parquet('../dashboard/data/df_sto.parquet.gzip', compression='gzip')

In [36]:
# df_sto = pd.read_parquet('../dashboard/data/df_sto.parquet.gzip')
# df_sto.head()

,stockcode,price,contador
23,z8,97543.20,4256
4,x5,96714.24,4246
10,y3,96689.29,4235
13,y6,96338.44,4295
6,x7,96125.07,4278
